<p style="text-align: center; font-size: 300%"> Computational Finance </p>
<img src="img/ABSlogo.svg" alt="LOGO" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

# Binomial Trees

\begin{equation*}
\begin{array}{ccccl}
\begin{array}{c}
t=0 \\
i=0%
\end{array}
&  &
\begin{array}{c}
t=\delta t \\
i=1%
\end{array}
&  &
\begin{array}{c}
t=T=2\delta t \\
i=N=2%
\end{array}
\\
&  &  &  &  \\
&  &  &  & \quad S_{0}uu \\
&  &  & \nearrow &  \\
&  & S_{0}u &  &  \\
& \nearrow &  & \searrow &  \\
S_{0} &  &  &  & \quad S_{0}ud=S_{0}du \\
& \searrow &  & \nearrow &  \\
&  & S_{0}d &  &  \\
&  &  & \searrow &  \\
&  &  &  & \quad S_{0}dd%
\end{array}%
\end{equation*}


\begin{equation*}
\begin{array}{ccccl}
&  &  &  & C_{N}(uu) \\
&  &  & \nearrow &  \\
&  & C_{1}(u) &  &  \\
& \nearrow &  & \searrow &  \\
C_{0} &  &  &  & C_{N}(ud)=C_{N}(du) \\
& \searrow &  & \nearrow &  \\
&  & C_{1}(d) &  &  \\
&  &  & \searrow &  \\
&  &  &  & C_{N}(dd)%
\end{array}%
\end{equation*}

In [1]:
import numpy as np
def calltree_naive(S0, K, T, r, sigma, N):
    """
    European call price based on an N-step binomial tree.
    """
    deltaT = T/float(N)
    u=np.exp(sigma * np.sqrt(deltaT))
    d=1/u
    p=(np.exp(r*deltaT) - d)/(u-d)
    C=np.zeros([N+1,N+1])
    S=np.zeros([N+1,N+1])
    piu=np.exp(-r*deltaT)*p
    pid=np.exp(-r*deltaT)*(1-p)
    for i in xrange(N+1):
        for j in xrange(i, N+1):
            S[i,j]=S0*u**j*d**(2*i)
    for i in xrange(N+1):
        C[i,N]=max(0, S[i,N]-K)
    for j in xrange(N-1,-1,-1):
        for i in xrange(j+1):
            C[i,j] = piu * C[i,j+1] + pid * C[i+1,j+1]
    return  C[0,0]

In [2]:
S0=50.;K=50.;T=5.0/12;r=.1;sigma=.4;N=500;
calltree_naive(S0, K, T, r, sigma, N)

6.1139619792052535

In [3]:
%timeit calltree_naive(S0, K, T, r, sigma, N) #ipython magic for timing things

1 loop, best of 3: 176 ms per loop


In [4]:
def calltree_numpy(S0, K, T, r, sigma, N):
    """
    European call price based on an N-step binomial tree.
    """
    deltaT = T/float(N)
    u=np.exp(sigma * np.sqrt(deltaT))
    d=1/u
    p=(np.exp(r*deltaT) - d)/(u-d)
    piu=np.exp(-r*deltaT)*p
    pid=np.exp(-r*deltaT)*(1-p)
    C=np.zeros([N+1,N+1])
    S=S0*u**np.arange(N+1)*d**(2*np.arange(N+1)[:, np.newaxis]) #newaxis is needed for broadcasting
    S=np.triu(S) #keep only upper tr
    C[:,N]=np.maximum(0, S[:,N]-K) #note maximum in place of max
    for j in xrange(N-1,-1,-1):
        C[:j+1,j] = piu * C[:j+1,j+1] + pid * C[1:j+2,j+1]
    return  C[0,0]

In [8]:
#assert throws an error if False. np.allclose test if all elements of an array are almost zero
assert np.allclose(calltree_naive(S0, K, T, r, sigma, N), calltree_numpy(S0, K, T, r, sigma, N))
%timeit calltree_numpy(S0, K, T, r, sigma, N)

100 loops, best of 3: 4.23 ms per loop


In [6]:
from numba import jit
calltree_numba=jit(calltree_naive)

In [7]:
assert np.allclose(calltree_naive(S0, K, T, r, sigma, N), calltree_numba(S0, K, T, r, sigma, N))
%timeit calltree_numba(S0, K, T, r, sigma, N)

100 loops, best of 3: 4.29 ms per loop
